In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nn1 = 80
nn2 = 108
nn3 = 156
nn4 = 244
nn5 = 319
nn6 = 435
nn7 = 2000
nn8 = 500
nc = 10
bs=64
dev = "cuda"
lr = 0.0001

In [ ]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
y = y.astype(int)

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
train_dset = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
train_loader =  torch.utils.data.DataLoader(train_dset, batch_size=2500)
test_dset = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())
test_loader =  torch.utils.data.DataLoader(test_dset, batch_size=64)
X = None

In [ ]:
mod = torch.load("/content/drive/MyDrive/mod.pth")

<ipython-input-6-9d182e3d3c7a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mod = torch.load("/content/drive/MyDrive/mod.pth")


In [ ]:
class FCN(nn.Module):
  def __init__(self):
    super(FCN, self).__init__()
    self.cc1 = nn.Conv2d(1, nn1, 5)
    self.cc2 = nn.Conv2d(nn1, nn2, 5)
    self.cc3 = nn.Conv2d(nn2, nn3, 5)
    self.cc4 = nn.Conv2d(nn3, nn4, 5)
    self.cc5 = nn.Conv2d(nn4, nn5, 3)
    self.cc6 = nn.Conv2d(nn5, nn6, 3)
    self.fc1 = nn.Linear(27840, nn7)
    self.fc2 = nn.Linear(nn7, nn8)
    self.fc3 = nn.Linear(nn8, nc)

  def forward(self, x):
    z1 = F.relu(self.cc1(x))
    z2 = F.relu(self.cc2(z1))
    z3 = F.relu(self.cc3(z2))
    z4 = F.relu(self.cc4(z3))
    z5 = F.relu(self.cc5(z4))
    z6 = F.relu(self.cc6(z5))
    #print(z6.shape)
    z6 = nn.Flatten()(z6)
    #print(z6.shape)
    z7 = F.dropout(F.relu(self.fc1(z6)), p=0.005)
    z8 = F.relu(self.fc2(z7))
    z9 = self.fc3(z8)
    return z9

In [ ]:
net = FCN()
net.to(dev)
net.load_state_dict(mod)
crit = nn.CrossEntropyLoss()
mod = None

In [ ]:
net.eval()
with torch.no_grad():
  cur, tot = 0, 0
  for a, b in test_loader:
    a = a.to(dev)
    b = b.to(dev)
    a = a.reshape(a.shape[0], 1, 28, 28)
    out = net(a)
    _, pred = torch.max(out, dim=1)
    tot+=b.shape[0]
    cur+=(pred==b).sum().item()
  print('accuracy:', cur/tot)

accuracy: 0.9941


In [ ]:
kernel = torch.tensor(
    [[[[1]]for i in range(nn6)] for j in range(1)], dtype=torch.float32).to(dev)
#kernel = kernel.reshape(1, 435, 3, 3)
conv = nn.Conv2d(in_channels=nn6, out_channels=1, kernel_size=1, bias=False).to(dev)
conv.weight = nn.Parameter(kernel)

kernel1 = torch.tensor(
    [[[[1]]for i in range(3)] for j in range(1)], dtype=torch.float32).to(dev)
#kernel = kernel.reshape(1, 435, 3, 3)
conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=1, bias=False).to(dev)
conv1.weight = nn.Parameter(kernel1)

In [ ]:
w = torch.nn.Sequential(net.cc1, nn.ReLU(), net.cc2, nn.ReLU(),  net.cc3, nn.ReLU(),  net.cc4, nn.ReLU(),  net.cc5, nn.ReLU(),  net.cc6, nn.ReLU(), nn.Flatten(), net.fc1, nn.ReLU())

In [ ]:
Xt=True
yt=True
we=0
for t, q in train_loader:
    t = t.to(dev)
    q = q.detach().numpy()
    t = t.reshape(t.shape[0], 1, 28, 28)
    out = w(t).to('cpu')
    #out = out.reshape(out.shape[0], out.shape[1]*out.shape[2]*out.shape[3])
    if type(Xt)!=bool:
      Xt = np.concatenate((Xt, out.detach().numpy()), axis=0)
      #print(out.shape, Xt.shape)
    else:
      Xt = out.detach().numpy()
    if type(yt)!=bool:
      yt = np.concatenate((yt, q))
    else:
      yt = q
    if Xt.shape[0]//6000==we:
      print(out.shape, Xt.shape)
      we+=1

torch.Size([2500, 2000]) (2500, 2000)
torch.Size([2500, 2000]) (7500, 2000)
torch.Size([2500, 2000]) (12500, 2000)
torch.Size([2500, 2000]) (20000, 2000)
torch.Size([2500, 2000]) (25000, 2000)
torch.Size([2500, 2000]) (30000, 2000)
torch.Size([2500, 2000]) (37500, 2000)
torch.Size([2500, 2000]) (42500, 2000)
torch.Size([2500, 2000]) (50000, 2000)
torch.Size([2500, 2000]) (55000, 2000)
torch.Size([2500, 2000]) (60000, 2000)


In [ ]:
Xt.shape, yt.shape

((60000, 2000), (60000,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(Xt, yt)

RandomForestClassifier(n_estimators=1000)

In [ ]:
#import joblib
#joblib.dump(rf, "for2.joblib")

In [ ]:
from google.colab import files
files.download('ranf.joblib')

FileNotFoundError: Cannot find file: ranf.joblib

In [ ]:
Xt=True
yt=True
we=0
for t, q in test_loader:
    t = t.to(dev)
    q = q.detach().numpy()
    t = t.reshape(t.shape[0], 1, 28, 28)
    out = w(t).to('cpu')
    #out = out.reshape(out.shape[0], out.shape[1]*out.shape[2]*out.shape[3])
    if type(Xt)!=bool:
      Xt = np.concatenate((Xt, out.detach().numpy()), axis=0)
      #print(out.shape, Xt.shape)
    else:
      Xt = out.detach().numpy()
    if type(yt)!=bool:
      yt = np.concatenate((yt, q))
    else:
      yt = q
    if Xt.shape[0]//6000==we:
      print(out.shape, Xt.shape)
      we+=1

In [ ]:
Xt.shape, yt.shape

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(rf.predict(Xt), yt))

In [ ]:
u = rf.predict(Xt)
op = X_test[y_test!=u]

In [ ]:
op.shape

In [ ]:
tt=0
plt.imshow(op[tt].reshape((28, 28)))
plt.plot()